In [ ]:
import functions as func
import pandas as pd
import numpy as np
import gensim
import re

from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from collections import defaultdict
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

# Preprocessing

In [ ]:
# Um diretorio que contém todos os datasets
path = 'data'

df_dict = func.readCSV_DATA(path)

In [ ]:
df_train = df_dict['train.csv']

In [ ]:
df_train.head()

In [ ]:
question_lst = []
answer_lst = []

for column in df_train.columns[11:]:
    match = re.match('([a-z]*)_', column)
    if match.group(1) == 'question':
        question_lst.append(column)
    else:
        answer_lst.append(column)

In [ ]:
df_train = func.prepareData(df_train)

# Training word2vec

In [ ]:
corpus = [word_tokenize(i) for i in df_train.text_all_clean]

In [ ]:
model = gensim.models.Word2Vec(corpus,
                               size=150,
                               window=5,
                               min_count=2,
                               workers=10)

In [ ]:
vocab = defaultdict(lambda : np.zeros(150))
for w in tqdm(list(model.wv.vocab.keys())):
    vocab[w] = model.wv[w]

In [ ]:
vec_docs = [[] for i in range(df_train.shape[0])]

for i, question_text in tqdm(enumerate(df_train.question_text_clean), total=df_train.shape[0]):
    for word in word_tokenize(question_text):
        vec_docs[i].append(vocab[word])